In [5]:
import numpy as np
from pyscf import gto
from pyscf import dft

mol = gto.Mole()
mol.atom = """
H     -0.70000   0.000000   0.000000
H     0.700000   0.000000   0.000000
C     0.000000   1.000000   0.000000
"""
mol.basis = "6-31G"
mol.spin = 2
mol.build()

#
# First converge a high-spin UKS calculation
#
mf = dft.UKS(mol)
mf.xc = "lda"
mf.kernel()

converged SCF energy = -38.1903445542632  <S^2> = 2.0256412  2S+1 = 3.0170457


np.float64(-38.190344554263234)

In [6]:
fock_real = mf.get_fock()

eri_ao = mf.mol.intor("int2e")
dm1 = mf.make_rdm1()
fock_check = (
    mf.mol.intor("int1e_kin")
    + mf.mol.intor("int1e_nuc")
    + np.einsum("uvkl, ykl -> uv", eri_ao, dm1)
)
fock_check = np.array([fock_check, fock_check])

In [7]:
from pyscf.dft import numint

ni = dft.numint.NumInt()
dm1 = mf.make_rdm1()

n, exc, vxc = ni.nr_uks(mol, mf.grids, mf.xc, dm1)
vxc_fake = np.zeros_like(vxc)
n_fake = np.zeros_like(n)
exc_fake = np.zeros_like(n)
n_fake[0], exc_fake[0], vxc_fake[0, :, :] = ni.nr_rks(mol, mf.grids, mf.xc, dm1[0] * 2)
n_fake[1], exc_fake[1], vxc_fake[1, :, :] = ni.nr_rks(mol, mf.grids, mf.xc, dm1[1] * 2)

In [8]:
print(n - n_fake / 2, exc - (exc_fake[0] + exc_fake[1]) / 2)

print(np.linalg.norm(vxc - vxc_fake))

[0. 0.] 0.0
5.698700191845116e-16


In [11]:
np.linalg.norm(fock_check + vxc_fake - fock_real)

np.float64(8.756054388158624e-15)

In [20]:
import scipy.linalg as LA
mat_s = mol.intor("int1e_ovlp")
mat_hs = LA.fractional_matrix_power(mat_s, -0.5).real

nocc = mol.nelec

eigvecs_inv, mo_inv = np.linalg.eigh(mat_hs @ (fock_real[0]) @ mat_hs)
mo_inv = mat_hs @ mo_inv
dm1_inv0 = mo_inv[:, : nocc[0]] @ mo_inv[:, : nocc[0]].T

eigvecs_inv, mo_inv = np.linalg.eigh(mat_hs @ (fock_real[1]) @ mat_hs)
mo_inv = mat_hs @ mo_inv
dm1_inv1 = mo_inv[:, : nocc[1]] @ mo_inv[:, : nocc[1]].T

dm1_inv = np.array([dm1_inv0, dm1_inv1])

In [21]:
np.linalg.norm(dm1_inv - mf.make_rdm1())

np.float64(3.276807649598094e-06)

In [1]:
from pyscf import gto, scf

mol = gto.M(atom="C 0 0 0; O 0 0 1.2", basis="631g*")
mf = scf.RHF(mol).run()
# Run stability analysis for the SCF wave function
mf.stability()

#
# This instability is associated to a symmetry broken answer.  When point
# group symmetry is used, the same wave function is stable.
#
mol = gto.M(atom="C 0 0 0; O 0 0 1.2", basis="631g*", symmetry=1)
mf = scf.RHF(mol).run()
mf.stability()

#
# In the RHF method, there are three kinds of instability: the internal
# instability for the optimal solution in RHF space and the external
# instability (including the RHF -> UHF instability, and real -> complex
# instability).  By default the stability analysis only detects the internal
# instability.  The external instability can be enabled by passing keyword
# argument external=True to the stability function.
#
mf.stability(external=True)

#
# If the SCF wavefunction is unstable, the stability analysis program can
# transform the SCF wavefunction and generate a set of initial guess (orbitals).
# The initial guess can be used to make density matrix and fed into a new SCF
# iteration (see 15-initial_guess.py).
#
mol = gto.M(atom="O 0 0 0; O 0 0 1.2222", basis="631g*")
mf = scf.UHF(mol).run()
mo1 = mf.stability()[0]
dm1 = mf.make_rdm1(mo1, mf.mo_occ)
mf = mf.run(dm1)
mf.stability()

#
# Also the initial guess orbitals from stability analysis can be used as with
# second order SCF solver
#
mf = mf.newton().run(mo1, mf.mo_occ)
mf.stability()

#
# The UHF method has the internal
# instability for the optimal solution in UHF space and the external
# instability (UHF -> GHF, and real -> complex).  By default the stability
# analysis only detects the internal instability.  The external instability
# can be enabled by passing keyword argument external=True to the stability
# function.
#
mf.stability(external=True)

converged SCF energy = -112.720258006549
<class 'pyscf.scf.hf.RHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -112.720258006549
<class 'pyscf.scf.hf_symm.SymAdaptedRHF'> wavefunction is stable in the internal stability analysis
<class 'pyscf.scf.hf_symm.SymAdaptedRHF'> wavefunction is stable in the internal stability analysis
<class 'pyscf.scf.hf_symm.SymAdaptedRHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.scf.hf_symm.SymAdaptedRHF'> wavefunction is stable in the RHF/RKS -> UHF/UKS stability analysis
converged SCF energy = -149.580876158935  <S^2> = 1.0205769  2S+1 = 2.2543973
<class 'pyscf.scf.uhf.UHF'> wavefunction has an internal instability.
converged SCF energy = -149.583978275383  <S^2> = 1.0162688  2S+1 = 2.2505722
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -149.583978275371  <S^2> = 1.016269  2S+1 = 2.2505724
<class 'pyscf.soscf.newton_

(array([[[-1.69738693e-02, -9.95875374e-01,  1.40542559e-01,
          -1.69541934e-01,  1.34305535e-07, -6.00059030e-02,
           6.00272309e-08, -2.56480677e-08,  1.41876144e-07,
           9.70084090e-02,  2.71850017e-02, -2.64179076e-07,
          -3.09145263e-02,  7.22971332e-07, -6.58787387e-02,
          -5.93082155e-07,  1.94538216e-08, -2.45949739e-02,
          -7.60131695e-09,  3.48977841e-09, -8.13377709e-09,
          -3.57426563e-03,  5.41820079e-09,  6.61820177e-03,
          -1.92960514e-02, -6.32289803e-10,  6.08710541e-09,
           7.21956495e-02],
         [-1.29887559e-04, -2.04455277e-02, -3.19214439e-01,
           4.02970068e-01, -3.14689716e-07,  1.36478469e-01,
          -1.25861522e-07,  1.00266067e-07, -2.43295758e-07,
          -2.12945245e-01, -2.63944151e-02,  1.01272205e-06,
           1.40621367e-01, -5.74992955e-06,  9.45540244e-01,
           8.43993867e-06, -3.07582599e-07,  1.28928377e+00,
           2.81009783e-07, -2.14058042e-08,  1.31295520e-